In [45]:
import tifffile
from cellpose import models
import numpy as np
from cellstitch.pipeline import *
from cellstitch.evaluation import *
import pandas as pd

In [52]:
dataset = "Sepal"

In [53]:
flow_threshold = 1
model = models.Cellpose(model_type='cyto')

###  Generate cellpose3d results

In [54]:
for i in range(50): 
    test_filename = "%s_%02d" % (dataset, i)
    print("Starting %s" % test_filename)
    img = tifffile.imread("../DATA/%s/%s.tif" % (dataset, test_filename))
    masks, _, _, _ = model.eval(img, do_3D=True, flow_threshold=flow_threshold)
    np.save("./results/%s/cellpose3d/%s.npy" % (dataset, test_filename), masks) 

Starting Sepal_00
Starting Sepal_01
Starting Sepal_02
Starting Sepal_03
Starting Sepal_04
Starting Sepal_05
Starting Sepal_06
Starting Sepal_07
Starting Sepal_08
Starting Sepal_09
Starting Sepal_10
Starting Sepal_11
Starting Sepal_12
Starting Sepal_13
Starting Sepal_14
Starting Sepal_15
Starting Sepal_16
Starting Sepal_17
Starting Sepal_18
Starting Sepal_19
Starting Sepal_20
Starting Sepal_21
Starting Sepal_22
Starting Sepal_23
Starting Sepal_24
Starting Sepal_25
Starting Sepal_26
Starting Sepal_27
Starting Sepal_28
Starting Sepal_29
Starting Sepal_30
Starting Sepal_31
Starting Sepal_32
Starting Sepal_33
Starting Sepal_34
Starting Sepal_35
Starting Sepal_36
Starting Sepal_37
Starting Sepal_38
Starting Sepal_39
Starting Sepal_40
Starting Sepal_41
Starting Sepal_42
Starting Sepal_43
Starting Sepal_44
Starting Sepal_45
Starting Sepal_46
Starting Sepal_47
Starting Sepal_48
Starting Sepal_49


## Generate cellstitch results

In [ ]:
for i in range(50): 
    test_filename = "%s_%02d" % (dataset, i)
    print("Starting %s" % test_filename)
    img = tifffile.imread("../DATA/%s/%s.tif" % (dataset, test_filename))
    
    cellstitch, _, _, _ = model.eval(list(img), flow_threshold=flow_threshold)
    cellstitch = np.array(cellstitch)

    yz_masks, _, _, _ = model.eval(list(img.transpose(1,0,2)), flow_threshold=flow_threshold)
    yz_masks = np.array(yz_masks).transpose(1,0,2)

    xz_masks, _, _, _ = model.eval(list(img.transpose(2,1,0)), flow_threshold=flow_threshold)
    xz_masks = np.array(xz_masks).transpose(2,1,0)

    full_stitch(cellstitch, yz_masks, xz_masks)
    
    np.save("./results/%s/cellstitch/%s.npy" % (dataset, test_filename), cellstitch)  

Starting Sepal_00
Starting Sepal_01
Starting Sepal_02
Starting Sepal_03
Starting Sepal_04
Starting Sepal_05
Starting Sepal_06
Starting Sepal_07
Starting Sepal_08
Starting Sepal_09


Starting Sepal_10


Starting Sepal_11
Starting Sepal_12
Starting Sepal_13
Starting Sepal_14
Starting Sepal_15
Starting Sepal_16
Starting Sepal_17
Starting Sepal_18


Starting Sepal_19
Starting Sepal_20
Starting Sepal_21
Starting Sepal_22


Starting Sepal_23
Starting Sepal_24
Starting Sepal_25
Starting Sepal_26
Starting Sepal_27


# Benchmark results

In [ ]:
ap_threshold = 0.5

In [ ]:
methods = ["cellstitch", "cellpose3d"]

for method in methods: 
    data = [] 
    for i in range(50): 
        filename = "%s_%02d" % (dataset, i)
        labels = tifffile.imread('../DATA/%s/%s_mask.tif' % (dataset, filename))
        true_num_cells = np.unique(labels).size - 1 
        true_avg_vol = get_avg_vol(labels) 
    
        masks = np.load("./results/%s/%s/%s.npy" % (dataset, method, filename))

        num_cells = np.unique(masks).size - 1
        d_num_cells = abs(num_cells - true_num_cells) / true_num_cells

        avg_vol = get_avg_vol(masks)
        d_avg_vol = abs(true_avg_vol - avg_vol) / true_avg_vol

        ap, tp, fp, fn = average_precision(labels, masks, ap_threshold)
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
    
        row = [
            filename, 
            d_num_cells, 
            d_avg_vol, 
            ap, 
            tp, 
            fp, 
            fn, 
            precision,
            recall, 
            true_num_cells, 
            true_avg_vol, 
            num_cells, 
            avg_vol
        ]

        data.append(row)

    df = pd.DataFrame(data, columns=[
        "filename",  
        "d_num_cells", 
        "d_avg_vol", 
        "ap", 
        "tp", 
        "fp", 
        "fn",
        "precision",
        "recall",
        "true_num_cells", 
        "true_avg_vol", 
        "num_cells", 
        "avg_vol"
    ])

    df.to_csv("./results/%s/%s.csv" % (dataset, method), index=False)

# Analyze results

In [35]:
method = 'cellpose3d'

In [36]:
df = pd.read_csv("./results/%s/%s.csv" % (dataset, method))

In [37]:
df.mean()

/tmp/ipykernel_584482/3698961737.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.mean()


d_num_cells           0.948993
d_avg_vol             0.975368
ap                    0.003825
tp                    0.640000
fp                  109.420000
fn                   63.880000
precision             0.006373
recall                0.011212
true_num_cells       64.200000
true_avg_vol      13797.459971
num_cells           109.740000
avg_vol             339.756656
dtype: float64